In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# read the data
titanic = pd.read_csv('/kaggle/input/titanic/train.csv')

# get a glimpse over the data
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# reviewing the data, I find out that all the numerical columns 
# are complete, except for the Age column, which is normal, 
# since some people may have sneaked into the boat to immigrate to America
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
# since Cabin was an object, it didn't appear in the describe method,
# but now we know it has a lot of null values
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
# the Cabin column has 147 different unique values,
# which makes one hot encoding inefficient
titanic['Cabin'].nunique()

147

In [6]:
# the Ticket column just tells you the 
# type of ticket, as "STON/02." may show.
# ordinal encoding may be useful here
titanic['Ticket']

0             A/5 21171
1              PC 17599
2      STON/O2. 3101282
3                113803
4                373450
             ...       
886              211536
887              112053
888          W./C. 6607
889              111369
890              370376
Name: Ticket, Length: 891, dtype: object

In [7]:
# in the data description of the competition,
# it says there are 3 ports, 
# C = Cherbourg
# Q = Queenstown
# S = Southampton
# this is perfect for one hot encoding
titanic['Embarked']

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

In [8]:
# I will drop this column, since it's useless
titanic['PassengerId']

0        1
1        2
2        3
3        4
4        5
      ... 
886    887
887    888
888    889
889    890
890    891
Name: PassengerId, Length: 891, dtype: int64

In [9]:
# I will create a new feature for the amount of family members
# family members = siblings + spouses + parents + children
titanic['FamMembers'] = titanic['SibSp'] + titanic['Parch']

In [10]:
# splitting the data into training and validation sets
from sklearn.model_selection import train_test_split

y = titanic['Survived']
X = titanic.drop('Survived', axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [11]:
# I drop PassengerId column
X_train.drop('PassengerId', axis=1, inplace=True)
X_valid.drop('PassengerId', axis=1, inplace=True)

In [12]:
# separating the data between numerical and categorical columns.
# since the dataset doesn't have too many columns, I will keep all
numerical_cols = X_train.select_dtypes(include=['int64','float64']).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

In [13]:
# we obtain One Hot Encoding, Ordinal Encoding and Target Encoding columns
ohe_cols = [col for col in categorical_cols if X_train[col].nunique() <= 3]
ordinal_cols = [col for col in categorical_cols if 3 < X_train[col].nunique() < 10]
high_cardinality_cols = [col for col in categorical_cols if X_train[col].nunique() >= 10]

In [14]:
# fill null values with the median in numerical columns
from sklearn.impute import SimpleImputer

num_cols_transformer = SimpleImputer(strategy='median')

In [15]:
# pipelines for categorical columns
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from category_encoders import TargetEncoder
from sklearn.pipeline import Pipeline

ohe_cols_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore',
                           sparse_output=False))
])

ordinal_cols_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value',
                                unknown_value=-1))
])

high_cardinality_cols_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('target', TargetEncoder())
])

In [16]:
# bundle all the pipelines into a ColumnTransformer
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_cols_transformer, numerical_cols),
        ('ohe', ohe_cols_transformer, ohe_cols),
        ('ord', ordinal_cols_transformer, ordinal_cols),
        ('hcc', high_cardinality_cols_transformer, high_cardinality_cols)
    ]
)

In [17]:
# prepare the data for pytorch
import torch
from sklearn.preprocessing import StandardScaler

X_train = preprocessor.fit_transform(X_train, y_train)
X_valid = preprocessor.transform(X_valid)

scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_valid_scaled = scaler_X.transform(X_valid)

X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_valid_tensor = torch.tensor(X_valid_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
y_valid_tensor = torch.tensor(y_valid.values, dtype=torch.float32)

In [30]:
# logistic regression model using SGD
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

train_ds = TensorDataset(X_train_tensor, y_train_tensor)
bs = 8 # batch size
train_dl = DataLoader(train_ds, bs, shuffle=True)

# define the neural network
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 1),
    nn.Sigmoid()
)

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# define loss function as Binary Cross-Entropy Loss
loss_fn = nn.BCELoss()

# the training loop
num_epochs = 100
for epoch in range(num_epochs):
    for x, y in train_dl:
        # produce the predictions
        predictions = model(x).squeeze()
        
        # obtain the loss
        loss = loss_fn(predictions, y)

        # compute the gradients
        loss.backward()

        # clip the values so that the gradients dont explode
        torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=1.0)

        # update the model parameters
        optimizer.step()

        # clear the gradients
        optimizer.zero_grad()

    with torch.no_grad():
        predictions = model(X_train_tensor).squeeze()
        epoch_loss = loss_fn(predictions, y_train_tensor)
        print(f'Epoch: {epoch} Loss: {epoch_loss.item()}')

loss = loss_fn(model(X_valid_tensor).squeeze(), y_valid_tensor)
print(loss.item())

Epoch: 0 Loss: 0.7122110724449158
Epoch: 1 Loss: 0.6617990732192993
Epoch: 2 Loss: 0.6153187155723572
Epoch: 3 Loss: 0.5738932490348816
Epoch: 4 Loss: 0.5363302230834961
Epoch: 5 Loss: 0.5028159618377686
Epoch: 6 Loss: 0.4726055860519409
Epoch: 7 Loss: 0.4454127550125122
Epoch: 8 Loss: 0.42095792293548584
Epoch: 9 Loss: 0.3987632989883423
Epoch: 10 Loss: 0.3787113130092621
Epoch: 11 Loss: 0.3604706823825836
Epoch: 12 Loss: 0.3438483774662018
Epoch: 13 Loss: 0.3286518454551697
Epoch: 14 Loss: 0.314712792634964
Epoch: 15 Loss: 0.3018898367881775
Epoch: 16 Loss: 0.2900587022304535
Epoch: 17 Loss: 0.27911677956581116
Epoch: 18 Loss: 0.26897624135017395
Epoch: 19 Loss: 0.25954434275627136
Epoch: 20 Loss: 0.25075361132621765
Epoch: 21 Loss: 0.24254514276981354
Epoch: 22 Loss: 0.23486192524433136
Epoch: 23 Loss: 0.22765477001667023
Epoch: 24 Loss: 0.22088457643985748
Epoch: 25 Loss: 0.21451182663440704
Epoch: 26 Loss: 0.20850302278995514
Epoch: 27 Loss: 0.20282907783985138
Epoch: 28 Loss: 0.1

In [31]:
# get the test predictions
titanic_test = pd.read_csv('/kaggle/input/titanic/test.csv')

titanic_test['FamMembers'] = titanic_test['SibSp'] + titanic_test['Parch']
X_test = titanic_test.drop('PassengerId', axis=1)
X_test = preprocessor.transform(X_test)
X_test_scaled = scaler_X.transform(X_test)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

model.eval()
with torch.no_grad():
    probabilities = model(X_test_tensor).squeeze()
    final_predictions = (probabilities > 0.5).int()

submission = pd.DataFrame({
    'PassengerId': titanic_test['PassengerId'],
    'Survived': final_predictions.numpy()
})

submission.to_csv('submission.csv', index=False)